In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv("log_order_parsing.csv")
df.fillna("",inplace=True)
del df['0']

In [2]:
x_train=[]
y_train=[]
df_len = len(df.columns)

In [3]:
for i in range(1,df_len):
    df = df.drop(df[df[str(i+1)]==""].index)
    x_train= x_train + df[str(i)].values.tolist()
    y_train= y_train + df[str(i+1)].values.tolist()
    del df[str(i)]
x_list = np.array(x_train)
y_list = np.array(y_train)

In [4]:
# one-hot encoding
import numpy as np
import tensorflow as tf

c_list = np.unique(x_train+y_train)
x_train = np.array(x_train)
y_train = np.array(y_train)

for i in range(len(x_train)):
    x_train[i] =  int(np.where(c_list==x_train[i])[0])
x_train = tf.keras.utils.to_categorical(x_train)
for j in range(len(y_train)):
    y_train[j] =  int(np.where(c_list==y_train[j])[0])
y_train = tf.keras.utils.to_categorical(y_train)

print(len(c_list))
print(c_list)
print(x_train.shape)

42
['4050' 'animalcompany' 'artwork-creative-h-and-m' 'artwork-dark' 'bj'
 'body' 'cat' 'ceo' 'club' 'couple' 'dog' 'event' 'evil-omen'
 'fashionshow' 'first-birthday' 'food' 'good-omen' 'group' 'kids'
 'landscape' 'location' 'lookbook' 'model-and-actor' 'musician' 'panorama'
 'person' 'portrait-strong' 'product' 'seminude' 'street' 'tour' 'uniform'
 'video-club' 'video-entertainment' 'video-excercise' 'video-fashion'
 'video-music' 'video-sketch' 'video-wedding' 'wedding' 'withyou' 'zoo']
(512, 42)


In [5]:
# Create Validation Data

# from sklearn.model_selection import train_test_split
# x_train,x_val,y_train,y_val = train_test_split(x_train,y_train,
#                                               test_size=0.2,
#                                               random_state=777)

In [6]:
#Create Test Data

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x_train,y_train,
                                              test_size=0.1,
                                              random_state=777)

In [7]:
#데이터가 복잡하지 않고 데이터의 수가 적은데 모델을 깊게 쌓으면 과대적합 발생률 증가

#CREATE MODEL
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
model = Sequential()

# model.add(tf.keras.layers.Embedding(len(c_list),len(c_list)))
model.add(Dense(64,activation="relu",input_shape=(len(c_list),)))
model.add(Dense(32,activation="relu"))
model.add(Dense(16,activation="relu"))



model.add(Dense(len(c_list),activation="softmax"))


model.summary()

# model.add(Dense(16,activation="relu"))

model.compile(optimizer=Adam(learning_rate=1e-3),loss="categorical_crossentropy",metrics=['acc'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                2752      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_3 (Dense)              (None, 42)                714       
Total params: 6,074
Trainable params: 6,074
Non-trainable params: 0
_________________________________________________________________


In [8]:
#학습과정
history = model.fit(x_train,y_train,
         epochs=100,
#           validation_data=(x_val,y_val)
         )

Epoch 1/100
15/15 [==============================] - 0s 732us/step - loss: 3.7110 - acc: 0.0848
Epoch 2/100
15/15 [==============================] - 0s 669us/step - loss: 3.6430 - acc: 0.1848
Epoch 3/100
15/15 [==============================] - 0s 733us/step - loss: 3.5531 - acc: 0.1891
Epoch 4/100
15/15 [==============================] - 0s 733us/step - loss: 3.4207 - acc: 0.1761
Epoch 5/100
15/15 [==============================] - 0s 667us/step - loss: 3.2446 - acc: 0.1587
Epoch 6/100
15/15 [==============================] - 0s 666us/step - loss: 3.1050 - acc: 0.1543
Epoch 7/100
15/15 [==============================] - 0s 667us/step - loss: 3.0355 - acc: 0.1609
Epoch 8/100
15/15 [==============================] - 0s 666us/step - loss: 2.9843 - acc: 0.1848
Epoch 9/100
15/15 [==============================] - 0s 600us/step - loss: 2.9438 - acc: 0.1913
Epoch 10/100
15/15 [==============================] - 0s 667us/step - loss: 2.9068 - acc: 0.1935
Epoch 11/100
15/15 [===================

In [9]:
# pred = model.predict(x_test)

#     print(c_list[np.argmax(x_test[idx])],c_list[np.argmax(pred[idx])],c_list[np.argmax(y_test[idx])])


In [10]:
def statistic_result(category):
    temp_list=np.array([])
    count_list =np.array([])
    for i in range(len(x_list)):
        if x_list[i]==category:
            temp_list=np.append(temp_list,y_list[i])
    for i in range(len(temp_list)):
        count_list =np.append(count_list,(temp_list == temp_list[i]).sum())
    
    result=temp_list[np.where(count_list.max())[0]]
    
#     print(result,count_list.max())
    return result
                                    

In [11]:

pred = model.predict(x_test)
print(x_test.shape)
cnt =0
for idx in range(len(pred)):
    key = c_list[np.argmax(x_test[idx])]
    print("현재방문 : {} 모델 예측값 : {} 통계적 예측값 : {} 실제 기록된 값 : {}".format(key
                                                                    ,c_list[np.argmax(pred[idx])]
                                                                    ,statistic_result(key)
                                                                    ,c_list[np.argmax(y_test[idx])]))
    if c_list[np.argmax(pred[idx])] == statistic_result(key) :
        cnt+=1

print(cnt)

(52, 42)
현재방문 : wedding 모델 예측값 : person 통계적 예측값 : ['video-wedding'] 실제 기록된 값 : video-excercise
현재방문 : zoo 모델 예측값 : zoo 통계적 예측값 : ['zoo'] 실제 기록된 값 : food
현재방문 : seminude 모델 예측값 : body 통계적 예측값 : ['body'] 실제 기록된 값 : person
현재방문 : dog 모델 예측값 : dog 통계적 예측값 : ['dog'] 실제 기록된 값 : dog
현재방문 : lookbook 모델 예측값 : product 통계적 예측값 : ['ceo'] 실제 기록된 값 : body
현재방문 : evil-omen 모델 예측값 : artwork-dark 통계적 예측값 : ['artwork-dark'] 실제 기록된 값 : lookbook
현재방문 : body 모델 예측값 : seminude 통계적 예측값 : ['seminude'] 실제 기록된 값 : body
현재방문 : video-fashion 모델 예측값 : product 통계적 예측값 : ['video-fashion'] 실제 기록된 값 : video-excercise
현재방문 : kids 모델 예측값 : seminude 통계적 예측값 : ['seminude'] 실제 기록된 값 : body
현재방문 : video-fashion 모델 예측값 : product 통계적 예측값 : ['video-fashion'] 실제 기록된 값 : video-music
현재방문 : seminude 모델 예측값 : body 통계적 예측값 : ['body'] 실제 기록된 값 : evil-omen
현재방문 : dog 모델 예측값 : dog 통계적 예측값 : ['dog'] 실제 기록된 값 : cat
현재방문 : model-and-actor 모델 예측값 : person 통계적 예측값 : ['person'] 실제 기록된 값 : seminude
현재방문 : street 모델 예측값 : person 통계적 예측값 : ['b